In [50]:
import numpy as np
import os
import random

job_slow_queries = [
    '6d.sql', 
    '6f.sql', 
    '8c.sql', '9d.sql', 
    '10c.sql', 
    '16b.sql', 
    '17a.sql', '17b.sql',  
    '17c.sql', 
    '17d.sql',  
    '17e.sql', 
    '17f.sql',  
    '18c.sql', '19c.sql', '19d.sql', 
    '20a.sql', 
    '25c.sql', '26c.sql', 
    '30a.sql'
]

job_rand_queries = [
    '8a.sql', '16a.sql', '2a.sql', '30c.sql', '17e.sql', '20a.sql', '26b.sql',
    '12b.sql', '15b.sql', '15d.sql', '10b.sql', '15a.sql', '4c.sql', '4b.sql',
    '22b.sql', '17c.sql', '24b.sql', '10a.sql', '22c.sql'
]


query_dir = './job/'
queries = job_slow_queries + job_rand_queries
query_set = set()
[query_set.add(q) for q in queries]
queries = list(query_set)

fact_tables = ['ci', 't', 'mc', 'mk', 'aka_t', 'mi', 'mi_idx', 'cc']

JOB_join_graph = {
    'ci': ['pi', 'ml', 'n', 'n1', 'rt', 'chn', 'an', 'an1', 'a1', 't', 'mc', 'aka_t', 'mi', 'mi_idx', 'cc'],
    't': ['ml', 'kt'] + ['ci', 'mc', 'mi', 'mi_idx', 'aka_t', 'at', 'mk', 'cc', 'mi_idx'],
    'mc': ['ct', 'cn'] + ['ci', 't', 'mk', 'aka_t', 'mi', 'mi_idx', 'cc'],
    'mk': ['k'] + ['ci', 't', 'mc', 'aka_t', 'mi', 'mi_idx', 'cc'],
    'cc': ['cct1', 'cct2'] + ['ci', 't', 'mc', 'mk', 'aka_t', 'mi', 'mi_idx'],
    'mi_idx': ['it', 'it2'] + ['ci', 't', 'mc', 'mk', 'aka_t', 'mi', 'cc'],
    'mi': ['it', 'it1'] + ['ci', 't', 'mc', 'mk', 'aka_t', 'mi_idx', 'cc'],
    'aka_t': ['t'],
    'at': ['t'],
    'chn': ['ci'],
    'an': ['ci'],
    'an1': ['ci'],
    'rt': ['ci'],
    'n': ['ci', 'pi', 'an', 'an1', 'a1'],
    'n1': ['ci', 'pi', 'an', 'an1', 'a1'],
    'ml': ['ci'],
    'pi': ['an', 'an1', 'n', 'n1', 'ci', 'it3'],
    'lt': ['ml'],
    'ml': ['t', 'lt'],
    'kt': ['t'],
    'k': ['mk'],
    'cct1': ['cc'],
    'cct2': ['cc'],
    'it': ['mi_idx', 'mi'],
    'it1': ['mi'],
    'it2': ['mi_idx'],
    'ct': ['mc'],
    'cn': ['mc'],
    'a1': ['n', 'ci', 'n1']
}


In [52]:
for q_file in queries:
    print(q_file)
    for r in range(1,5):
        with open(query_dir + q_file, 'r') as f:
            original_sql = f.read()
        tables = original_sql.replace('\n', ' ').split('FROM')[-1].split('WHERE')[0].split(',')
        tables = [t.split('AS')[1].replace(' ', '') for t in tables]
        q_fact_tables = [t for t in tables if t in fact_tables]
        random.shuffle(q_fact_tables)
        q_dim_tables = [t for t in tables if t not in fact_tables]
        jo = []
        while len(jo) < len(tables):
            if len(jo) == 0:
                jo.append(random.sample(q_fact_tables, 1)[0])
            else:
                available_next_tabs = []
                for t in jo:
                    for t_neighbor in JOB_join_graph[t]:
                        if t_neighbor in tables and t_neighbor not in jo:
                            available_next_tabs.append(t_neighbor)
                assert len(available_next_tabs) > 0, f"{jo}, {tables}"
                next_table = random.sample(available_next_tabs, 1)[0]
                jo.append(next_table)
        jo_str = f'({jo[0]} {jo[1]})'
        for t in jo[2:]:
            jo_str = '(' +  jo_str + ' ' + t + ')'
        jo_str = 'Leading(' + jo_str + ')'
        pysical_op_str = ''
        for i in range(2, len(jo)+1):
            pysical_op_str = f"HashJoin({' '.join(jo[:i])})\n" + pysical_op_str
#         print(pysical_op_str)
#         print(jo_str)
        
        pg_plan = f'/*+\n{jo_str}*/'
        adaptive_plan = f'/*+\n{pysical_op_str}{jo_str}*/'
        
        pg_target_query_dir = f'./job_rand_plan/rand_{r}/'
#         os.system(f'mkdir {pg_target_query_dir}')
        with open(pg_target_query_dir + q_file, 'w') as g:
            g.write(pg_plan + '\n' + original_sql)
        
        
#         aqo_target_query_dir = f'./job_lip_rand_plan/rand_{r}/'
# #         os.system(f'mkdir {aqo_target_query_dir}')
#         with open(aqo_target_query_dir + q_file, 'w') as g:
#             g.write(adaptive_plan + '\n' + original_sql)
#     break


25c.sql
26c.sql
10a.sql
4c.sql
19d.sql
10b.sql
15d.sql
24b.sql
30a.sql
17c.sql
4b.sql
16b.sql
8a.sql
17d.sql
15a.sql
12b.sql
17a.sql
6f.sql
2a.sql
16a.sql
22c.sql
17b.sql
22b.sql
19c.sql
17f.sql
30c.sql
8c.sql
17e.sql
20a.sql
18c.sql
26b.sql
15b.sql
6d.sql
10c.sql
9d.sql
